In [ ]:
import time

# ITEMS

bench = {
    "name": "bench",
    "type": "furniture",
    "icon" : "■",
    "location" : (6,3), # item and player coordinates are room based, there are no map coordinates
}

locker = {
    "name": "locker",
    "type": "furniture",
    "icon" : "●",
    "location" : (12,8),
}

shampoo = {
   "name": "shampoo",
   "type": "object",
   "icon" : "■",
   "location" : (6,3),
}

towel = {
    "name": "towel",
    "type": "object",
    "icon" : "●",
    "location" : (8,4),
}

desk = {
    "name": "desk",
    "type": "furniture",
    "icon" : "■",
    "location" : (6,3),
}

computer = {
    "name": "computer",
    "type": "object",
    "icon" : "●",
    "location" : (4,8),
}

safe = {
    "name": "safe",
    "type": "furniture",
    "icon" : "◆",
    "location" : (18,3),
}

box = {
    "name": "box",
    "type": "furniture",
    "icon" : "■",
    "location" : (2,8),
}

camera = {
    "name": "camera",
    "type": "object",
    "icon" : "●",
    "location" : (3,4),
}

painting = {
    "name": "painting",
    "type": "object",
    "icon" : "◆",
    "location" : (18,6),
}

trophy = {
    "name": "trophy",
    "type": "object",
    "icon" : "🏆",
    "location" : (9,4),
}


colors = {
    'PURPLE' : '\033[95m',
    'BLUE' : '\033[94m',
    'CYAN' : '\033[96m',
    'GREEN' : '\033[92m',
    'YELLOW' : '\033[93m',
    'RED' : '\033[91m',
    'ENDC' : '\033[0m',
    'BOLD' : '\033[1m',
    'UNDERLINE' : '\033[4m'
    }


# DOORS

door_a = {
    "name": "glass door",
    "type": "door",
}

door_b = {
    "name": "office door",
    "type": "door",
}

door_c = {
    "name": "double door",
    "type": "door",
}

door_d = {
    "name": "secure door",
    "type": "door",
}


# KEYS

key_a = {
    "name": "rusty key",
    "type": "key",
    "target": door_a,
}

key_b = {
    "name": "hair clip",
    "type": "key",
    "target": door_b,
}

key_c = {
    "name": "swiping card",
    "type": "key",
    "target": door_c,
}

key_d = {
    "name": "multi pass",
    "type": "key",
    "target": door_d,
}

# ROOMS

changing_room = {
    "name": "locker room",
    "type": "room",
}

shower_room = {
    "name": "shower room",
    "type": "room",
}

coach_office = {
    "name": "coach office",
    "type": "room",
}

press_room = {
    "name": "press room",
    "type": "room",
}

outside = {
  "name": "stadium",
  "type": "room",
}

# used to assign the correct rooms

rooms_layout = [["shower room", None, None],
                ["locker room", "coach office", None],
                [None, "press room", "stadium"]]


all_rooms = [changing_room, shower_room, coach_office, press_room, outside]

all_doors = [door_a, door_b, door_c, door_d]

# define which items/rooms are related

object_relations = {

    #rooms
    "locker room": [bench, locker, door_a, door_b],
    "shower room": [door_a, shampoo, towel],
    "coach office": [door_b, door_c, desk, computer, safe ],
    "press room": [door_c, door_d, painting, box, camera],
    "stadium": [door_d, trophy],

    #hiding places
    "locker": [key_a],
    "shampoo": [key_b],
    "safe": [key_c],
    "painting": [key_d],

    #doors
    "glass door": [changing_room, shower_room],
    "office door": [changing_room, coach_office],
    "double door": [press_room, coach_office],
    "secure door": [press_room, outside]
}

# define game state. Do not directly change this dict.
# Instead, when a new game starts, make a copy of this
# dict and use the copy to store gameplay state. This
# way you can replay the game multiple times.

INIT_GAME_STATE = {
    "current_room": changing_room,
    "keys_collected": [],
    "player_color": 'RED',
    "player_icon": "⚇",
    "player_location": (9,4),
    "target_room": outside,
    "max_time": 30 * (len(all_rooms) - 1) #30 seconds per room minus outside
}

In [ ]:
def linebreak():
    """
    Print a line break
    """
    print("\n\n")

def start_game():
    """
    Start the game
    """

    print(
    f"""
    You wake up on a {bench['name']} and find yourself on the floor of the {game_state['current_room']['name']}
    of your football nation. You don't remember what happened before but you can hear the crowd on fire. You start to panic, you are running late for the world cup final.
    You must get to the pitch, NOW! Or you will be kicked out of the team.
    """)

    title_screen()

    # initialize the timer
    if 'start_time' not in game_state :
      game_state['start_time'] = current_time()

    play_room(game_state["current_room"], True)


def play_room(room, redraw_map = False):

    """@
    Play a room. First check if the room being played is the target room.
    If it is, the game will end with success. Otherwise, let player either
    explore (list all items in this room) or examine an item found here.

    From object_relations, find the two rooms connected to the given door.

    Arguments:
    game_state (dictionnary): A dictionnary that describes your action.
    current_room (dictionnary): A dictionnary that describes a room.
    name (dictionnary): A dictionnary that describes a room.
    """
    game_state["current_room"] = room

    # prevents the map to be redrawn after any action
    if redraw_map:
        move_player()

    # lose condition if the timer is over
    if time_remaining(game_state['max_time']) < 0:
        lose_screen()
        print(f"{colors['RED']}YOU LOST. The game started without you and your team lost the World Cup.{colors['ENDC']}")
        return

    if(game_state["current_room"] == game_state["target_room"]):
        win_screen()
        print(f"{colors['GREEN']}AWESOME! You made it to the final with {time_remaining(game_state['max_time'])} seconds remaining! Your team won the world cup !{colors['ENDC']}")
        return
    else:
        print("You are now in " + room["name"])
        linebreak()
        print(f"YOU HAVE {colors['RED']}{time_remaining(game_state['max_time'])} SECONDS{colors['ENDC']} LEFT BEFORE THE FINAL")
        intended_action = input(f"What would you like to do? Type 'explore' or 'examine'? \n\n").strip().lower()
        if intended_action == "explore":
            explore_room(room)
            play_room(room)
        elif intended_action == "examine":
            examine_item(input("What would you like to examine? \n\n").strip().lower())
        else:
            print("Not sure what you mean. Type 'explore' or 'examine'.")
            play_room(room)
        linebreak()

def explore_room(room):
    """
    Explore a room. List all items belonging to this room.
    """
    items = [f"{colors['GREEN']}{i['name']}{colors['ENDC']}" for i in object_relations[room["name"]]]
    print(f"You explore the room. This is {room['name']}. You find " + ", ".join(items))

def get_next_room_of_door(door, current_room):
    """
    From object_relations, find the two rooms connected to the given door.

    Arguments:
    door (dictionnary): A dictionnary that describes the door.
    current_room (dictionnary): A dictionnary that describes a room.

    Returns:
    dictionnary: the room that is not the current_room.
    """
    connected_rooms = object_relations[door["name"]]
    for room in connected_rooms:
        if(not current_room == room):
            return room

def examine_item(item_name):
    """
    Examine an item which can be a door or furniture.
    First make sure the intended item belongs to the current room.
    Then check if the item is a door. Tell player if key hasn't been
    collected yet. Otherwise ask player if they want to go to the next
    room. If the item is not a door, then check if it contains keys.
    Collect the key if found and update the game state. At the end,
    play either the current or the next room depending on the game state
    to keep playing.
    """
    current_room = game_state["current_room"]
    next_room = ""
    output = None

    for item in object_relations[current_room["name"]]:
        if(item["name"] == item_name):

            if "location" in item: #ignores items without coordinates
                move_player(item["location"])

            output = "You examine " + item_name + ". "
            if(item["type"] == "door"):
                have_key = False
                for key in game_state["keys_collected"]:
                    if(key["target"] == item):
                        have_key = True
                if(have_key):
                    output += f"You unlock it with a {item['name']} you have."
                    next_room = get_next_room_of_door(item, current_room)
                else:
                    output += "It is locked but you don't have the key."
            else:
                if(item["name"] in object_relations and len(object_relations[item["name"]])>0):
                    item_found = object_relations[item["name"]].pop()
                    game_state["keys_collected"].append(item_found)
                    output += f"{colors['GREEN']}You find {item_found['name']}.{colors['ENDC']}"
                else:
                    output += f"{colors['RED']}There isn't anything interesting about it.{colors['ENDC']}"
            print(output)
            break

    if(output is None):
        print("The item you requested is not found in the current room.")

    if(next_room):
        play_room(next_room, True)
    else:
        play_room(current_room)


def win_screen():
  print(
        '''
               ⢀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⣀⠀⠀⠀⠀
          ⢠⣤⣤⣤⣼⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⣄⣤⣤⣠
          ⢸⠀⡶⠶⠾⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡷⠶⠶⡆⡼
          ⠈⡇⢷⠀⠀⣇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢰⠇⠀⢸⢁⡗
          ⠀⢹⡘⡆⠀⢹⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡸⠀⢀⡏⡼⠀
          ⠀⠀⢳⡙⣆⠈⣇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢠⠇⢀⠞⡼⠁⠀
          ⠀⠀⠀⠙⣌⠳⣼⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣞⡴⣫⠞⠀⠀⠀
          ⠀⠀⠀⠀⠈⠓⢮⣻⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⡼⣩⠞⠉⠀⠀⠀⠀
          ⠀⠀⠀⠀⠀⠀⠀⠉⠛⣆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢠⠞⠋⠁⠀⠀⠀⠀⠀⠀
          ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⢤⣀⠀⠀⠀⠀⢀⣠⠖⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀
          ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠉⡇⢸⡏⠉⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
          ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡇⢸⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
          ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢠⠖⠒⠓⠚⠓⠒⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
          ⠀⠀⠀⠀⠀⠀⠀⣀⣠⣞⣉⣉⣉⣉⣉⣉⣉⣉⣉⣉⣙⣆⣀⡀⠀⠀⠀⠀⠀⠀
          ⠀⠀⠀⠀⠀⠀⠀⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡇⠀⠀⠀⠀⠀⠀
          ⠀⠀⠀⠀⠀⠀⠀⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡇⠀⠀⠀⠀⠀⠀
          ⠀⠀⠀⠀⠀⠀⠀⠓⠲⠶⠶⠶⠶⠶⠶⠶⠶⠶⠶⠶⠶⠶⠖⠃⠀⠀⠀⠀⠀⠀


      	ᕕ( ᐛ )ᕗ
        '''
  )

def lose_screen():
  linebreak()
  linebreak()
  print(
        '''
        ヽ༼ಥ_ಥ༽ﾉ
        '''
  )

def title_screen():
    print(
'''
     _            _ _
    | |          | (_)
  __| |_ __ _  __| |_ _   _ _ __ ___
/ __| __/ _` |/ _` | | | | | '_ ` _ \
\__ \ || (_| | (_| | | |_| | | | | | |
|___/\__\__,_|\__,_|_|\__,_|_| |_| |_|
░▒▓███████▓▒░ ░▒▓██████▓▒░░▒▓███████▓▒░░▒▓█▓▒░░▒▓██████▓▒░
░▒▓█▓▒░░▒▓█▓▒░▒▓█▓▒░░▒▓█▓▒░▒▓█▓▒░░▒▓█▓▒░▒▓█▓▒░▒▓█▓▒░░▒▓█▓▒░
░▒▓█▓▒░░▒▓█▓▒░▒▓█▓▒░░▒▓█▓▒░▒▓█▓▒░░▒▓█▓▒░▒▓█▓▒░▒▓█▓▒░
░▒▓███████▓▒░░▒▓████████▓▒░▒▓█▓▒░░▒▓█▓▒░▒▓█▓▒░▒▓█▓▒░
░▒▓█▓▒░      ░▒▓█▓▒░░▒▓█▓▒░▒▓█▓▒░░▒▓█▓▒░▒▓█▓▒░▒▓█▓▒░
░▒▓█▓▒░      ░▒▓█▓▒░░▒▓█▓▒░▒▓█▓▒░░▒▓█▓▒░▒▓█▓▒░▒▓█▓▒░░▒▓█▓▒░
░▒▓█▓▒░      ░▒▓█▓▒░░▒▓█▓▒░▒▓█▓▒░░▒▓█▓▒░▒▓█▓▒░░▒▓██████▓▒░

'''
    )



In [ ]:
#mapping

# setup a single empty room
def create_room(room_name):
    room = []
    ceiling_tile = '-'
    wall_tile = '|'
    width = 20
    height = width // 2

    for i in range(height):

        if room_name == 'empty':
            room.append(' ' * width)
        else:
            if i == 0:
                room.append(room_name + ceiling_tile * (width - len(room_name)))
            elif i == height - 1:
                room.append(ceiling_tile * width)
            else:
                room.append(wall_tile + ' ' * (width - 2) + wall_tile)


    return room

# collects the relevant items and coordinates
def get_room_items(room_name):
  items = [item for item in object_relations[room_name] if "location" in item]
  items_locations = {item["icon"]:item["location"] for item in items}

  if game_state["current_room"]["name"] == room_name:
    player_avatar = colors[game_state['player_color']]+game_state['player_icon']+colors['ENDC']
    items_locations[player_avatar] = game_state['player_location']

  return items_locations

# slices the icons in the appropriate rooms
def insert_items(room, items):

    for icon, (x, y) in items.items():
        room[y] = room[y][:x] + icon + room[y][x + 1:]
    return room

# concatenates the rooms according to the layout
def create_map(layout):
    map = []
    for row in layout:
        row_of_rooms = []
        for room in row:
            if room is None:
                row_of_rooms.append(create_room('empty'))
            else:
                room_layout = create_room(room)
                items = get_room_items(room)
                furnished_room = insert_items(room_layout, items)
                row_of_rooms.append(furnished_room)
        map.append(row_of_rooms)
    return map

# updates the player location and the map
def move_player(location = INIT_GAME_STATE['player_location']):
  game_state['player_location'] = location
  display_map()

# prints the map to the output
def display_map():

    map = create_map(rooms_layout)

    final_map = ''

    for row_of_rooms in map:
        display_row = ''
        row_size = len(row_of_rooms[0])

        for i in range(row_size):
            for room in row_of_rooms:
                display_row += room[i]
            display_row += '\n'
        final_map += display_row

    linebreak()
    print(final_map)

#TIMER

# calculates the remaining time
def time_remaining(seconds):
    time_left =  seconds - (current_time() - game_state['start_time'] )
    return time_left

# gets the current time in seconds
def current_time():
    unix_time = int(time.time())
    return unix_time

# calculates the completion time
def completion_time():
    return (current_time() - game_state['start_time'])


In [ ]:
game_state = INIT_GAME_STATE.copy()

start_game()